libraries

In [1]:
%run import_modules.py  
%matplotlib inline

# For DNN modeling
import tensorflow as tf

# Tensorflow warning off
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.random.set_seed(2020)
import keras
from keras import backend as K
from keras.layers import * #Input, Dense
from keras.models import * #Model
from keras.optimizers import *
from keras.initializers import *
from keras.regularizers import *
from keras.utils.np_utils import *
from keras.utils.vis_utils import * #model_to_dot

Using TensorFlow backend.


load data

In [141]:
X_train, X_test, y_train, y_test, X_dep, ID_dep = pd.read_pickle('case3_train_test.pkl')
X_train

,3,62,6,4,1,2,29,17,15,13,...,34,20,54,31,42,8,43,12,10,18
849,0.548899,-0.211737,0.571544,0.359882,-1.015259,0.173466,0.183404,0.145489,-0.245481,-0.146008,...,-0.001879,0.463131,0.042479,0.118152,-0.014069,0.033457,-0.048870,-0.462602,-0.233664,0.028230
820,0.320871,-0.211737,0.056200,0.349123,-0.971234,-0.717802,-0.146723,0.081508,-0.222676,-0.225992,...,-0.022743,0.108870,0.028156,-0.060406,-0.022906,-0.630031,-0.069524,-0.145507,-0.335948,-0.199120
886,0.106787,-0.008796,0.836666,0.173059,1.202737,0.258409,-0.074363,0.048298,0.149960,0.461857,...,0.098704,-0.390307,0.050151,-0.247680,0.161154,0.333168,-0.140870,0.056745,0.719127,0.448463
1288,0.042551,-0.211737,0.423766,-0.014149,-1.347939,0.636159,-0.172775,-0.243410,-0.199761,0.531453,...,0.037228,-0.229491,0.176807,0.112770,0.052658,0.252694,-0.168742,-0.597824,0.560783,-0.126377
414,0.154722,0.787083,-0.567632,0.867433,0.464774,0.066818,-0.017657,0.337979,0.406158,-0.659014,...,-0.226443,-0.028724,-0.095207,0.045055,-0.016248,-0.867206,-0.181159,-0.327944,-0.049611,-0.236074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,0.147064,0.787083,-0.160119,-1.468848,0.211420,-0.198602,0.118466,-0.129578,-0.058548,-0.260203,...,-0.214193,0.327446,0.048855,-0.014029,0.004971,-0.773520,0.071237,-0.030003,0.458061,0.717578
3264,0.879341,0.787083,-0.245290,-1.022907,-0.232751,0.093136,0.178418,-0.038884,-0.036894,0.090965,...,0.167855,-0.163514,-0.030396,-0.278591,0.454730,-0.511544,0.062086,0.112773,0.229144,0.219065
1653,-0.195824,-0.579319,0.249590,0.050197,-1.242144,0.046300,-0.024794,0.169448,-0.137269,-0.072191,...,0.138992,-0.172225,0.106762,-0.225918,0.047244,0.100568,0.022075,-0.042563,0.410268,0.431392
2607,0.375166,-0.211737,-0.286549,0.105563,-1.155629,0.382463,0.244659,0.015792,0.059937,-0.107648,...,0.090222,-0.271835,-0.110292,0.077271,0.027430,0.306769,0.023316,-0.201401,-0.006521,-0.088862


In [140]:
X_train, X_test, y_train, y_test, X_dep, ID_dep = pd.read_pickle('case3_train_test.pkl')

X_train.shape, X_test.shape


seed = 2020
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

i = int(round(X_train.shape[0] * 0.8, 0))
X_valid, y_valid = X_train[i:], y_train[i:]
X_train, y_train = X_train[:i], y_train[:i]

model architecture

In [13]:
X_train

,3,62,6,4,1,2,29,17,15,13,...,34,20,54,31,42,8,43,12,10,18
849,0.548899,-0.211737,0.571544,0.359882,-1.015259,0.173466,0.183404,0.145489,-0.245481,-0.146008,...,-0.001879,0.463131,0.042479,0.118152,-0.014069,0.033457,-0.048870,-0.462602,-0.233664,0.028230
820,0.320871,-0.211737,0.056200,0.349123,-0.971234,-0.717802,-0.146723,0.081508,-0.222676,-0.225992,...,-0.022743,0.108870,0.028156,-0.060406,-0.022906,-0.630031,-0.069524,-0.145507,-0.335948,-0.199120
886,0.106787,-0.008796,0.836666,0.173059,1.202737,0.258409,-0.074363,0.048298,0.149960,0.461857,...,0.098704,-0.390307,0.050151,-0.247680,0.161154,0.333168,-0.140870,0.056745,0.719127,0.448463
1288,0.042551,-0.211737,0.423766,-0.014149,-1.347939,0.636159,-0.172775,-0.243410,-0.199761,0.531453,...,0.037228,-0.229491,0.176807,0.112770,0.052658,0.252694,-0.168742,-0.597824,0.560783,-0.126377
414,0.154722,0.787083,-0.567632,0.867433,0.464774,0.066818,-0.017657,0.337979,0.406158,-0.659014,...,-0.226443,-0.028724,-0.095207,0.045055,-0.016248,-0.867206,-0.181159,-0.327944,-0.049611,-0.236074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,-0.928741,-0.008151,0.016220,-0.431760,1.916867,0.652490,0.229067,0.269690,-0.114792,0.256507,...,0.522800,0.344829,-0.108672,-0.867108,-0.019721,0.069372,0.848311,-1.090585,-0.142489,-0.350349
603,0.502392,-0.579319,-0.471605,1.034412,-0.384312,0.414507,-0.114771,-0.019339,-0.156532,-0.177005,...,-0.088521,-0.335577,0.014870,0.083238,-0.034318,-0.616039,-0.175315,-0.376759,-0.349107,0.280511
1289,-1.185443,-0.008796,0.076312,0.237582,2.202188,-1.249115,-0.192562,1.473840,0.392467,-0.326854,...,-0.020377,-0.615013,0.076285,0.149394,0.144305,0.045316,-0.141914,-0.081162,0.613865,-0.108081
2502,0.007327,-0.008151,0.415069,0.829052,2.302148,-1.605445,-0.645921,0.013671,0.364496,-0.565396,...,0.099455,0.258509,-0.085870,-0.450350,-0.961301,-0.341927,0.119224,-0.433843,-0.456474,0.471285


In [139]:
dr1st = dense_range_1st_layer = [4, 8, 16, 32]
dr2nd = dense_range_2nd_layer = [4, 8, 16, 32]
dr3rd = dense_range_3rd_layer = [4, 8, 16, 32]
drop1st = dropout_range_1st_layer = [0, 0.2, 0.3, 0.4, 0.5]
drop2nd = dropout_range_2nd_layer = [0, 0.2, 0.3, 0.4, 0.5]

for dr1 in dr1st:
    
    for drop1 in drop1st:
    
        for dr2 in dr2nd:
        
            for drop2 in drop2nd:
                
                for dr3 in dr3rd:

        
# model architecture
                    model = Sequential(name = 'dnn model')
                    model.add(Dense(dr1, input_shape=(X_train.shape[1],), activation='relu'))
                    model.add(Dropout(drop1))
                    model.add(Dense(dr2, activation='relu'))
                    model.add(Dropout(drop2))
                    model.add(Dense(dr3, activation='relu'))
                    model.add(Dense(1, activation='sigmoid'))

# choose the optimizer and the cost function
                    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# model.summary()


# train the model -> verbose=0: silent
                    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=25)]

                    hist = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                        batch_size=64, epochs=200, callbacks=callbacks, verbose=0)

# visualize training history
# plt.plot(hist.history['loss'], label='train loss')
# plt.plot(hist.history['val_loss'], label='validation loss')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('Loss')
# plt.show()


# plt.plot(hist.history['acc'], label='train acc')
# plt.plot(hist.history['val_acc'], label='validation acc')
# plt.legend()
# plt.xlabel('epoch')
# plt.title('acc')
# plt.show()

# evaluate the model performance

#                    print(model.evaluate(X_test, y_test))
                    if roc_auc_score(y_test, model.predict(X_test)) >= 0.755:
                        print(f'1층: {dr1},드롭1: {drop1},2층: {dr2}, 드롭2: {drop2}, 3층: {dr3}')
                        print(roc_auc_score(y_test, model.predict(X_test)))

1층: 4,드롭1: 0,2층: 4, 드롭2: 0, 3층: 4
0.754451755845315
1층: 4,드롭1: 0,2층: 4, 드롭2: 0, 3층: 16
0.7520051984451639
1층: 4,드롭1: 0,2층: 8, 드롭2: 0.2, 3층: 8
0.7528840018632981
1층: 4,드롭1: 0,2층: 8, 드롭2: 0.3, 3층: 4
0.7529681634378633
1층: 4,드롭1: 0,2층: 8, 드롭2: 0.4, 3층: 8
0.7552542266725645
1층: 4,드롭1: 0,2층: 16, 드롭2: 0, 3층: 4
0.7533791850810887
1층: 4,드롭1: 0,2층: 16, 드롭2: 0.2, 3층: 8
0.7549488963090256
1층: 4,드롭1: 0,2층: 16, 드롭2: 0.3, 3층: 16
0.751797730377631
1층: 4,드롭1: 0,2층: 16, 드롭2: 0.4, 3층: 8
0.7521226332003712
1층: 4,드롭1: 0,2층: 32, 드롭2: 0, 3층: 4
0.7580433021087367
1층: 4,드롭1: 0,2층: 32, 드롭2: 0, 3층: 32
0.7507114588919639
1층: 4,드롭1: 0,2층: 32, 드롭2: 0.2, 3층: 4
0.7601042820626239
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 4
0.7557631106117959
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 8
0.7504276582335464
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 16
0.7534555176719734
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0, 3층: 32
0.7549919557192684
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0.3, 3층: 8
0.7572075581008451
1층: 4,드롭1: 0.2,2층: 4, 드롭2: 0.3, 3층: 16
0.7511988131260741
1층: 4

KeyboardInterrupt: 

predict unseen data

In [75]:
pred = model.predict(X_dep).flatten()
print(pred.shape)


(2482,)


In [77]:
# make submissions

pd.DataFrame({'cust_id': ID_dep, 'gender': pred}).to_csv('submission_dnn_nm.csv', index=False, encoding='cp949')
pd.read_csv('submission_dnn_nm.csv', encoding='cp949')

,cust_id,gender
0,3500,0.778295
1,3501,0.210400
2,3502,0.139749
3,3503,0.169812
4,3504,0.267438
...,...,...
2477,5977,0.516476
2478,5978,0.382308
2479,5979,0.816023
2480,5980,0.276791


save the model for future use

In [55]:
# pickle 형식으로 저장
with open('nm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [56]:
# 추후 저장한 모형 불러올 때
model = pd.read_pickle('nm_model.pkl')

In [58]:
# h5 형식으로 저장
model.save('nm_model.h5')

In [59]:
# 추후 저장한 모형 불러올 때
model = load_model('nm_model.h5')

1. libraries, read data     
2. data engineering   
3. model architecture, choose the optimizer and the cost function, train the model   
4. visualize training history, evaluate the model performance   
5. predict unseen data, save the model for future use   